<a href="https://colab.research.google.com/github/Ines8991/classification_rna/blob/main/Pr%C3%A9traitement_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_excel(r'/content/donnees.xlsx')
data.shape

(50862, 73)

In [3]:
data['Année_N'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

Traitement des valeurs manquantes

In [4]:
result = data.isna().sum().to_string(max_rows=None)
print(result)

Année_N                                            0
Individu - Code Etudiant                           0
Individu - Sexe                                    0
Nationalité (code)                                 0
Nationalité (lib.)                                 0
Nationalité - continent                            0
Age au bac                                       184
Bac ou équivalence (code)                          0
Groupe de bac                                      0
Série du bac                                       0
Bac - Mention (lib.)                            2688
Bac - Etab. préparation (code)                  8457
Code commune Lycée                             11172
IPS du lycée                                   11172
Etudiant oui ou oui-si                             0
PCS - parent (code)_Année_N                        0
PCS - parent (lib.)_Année_N                        0
PCS - autre parent (code)_Année_N                  1
Profession du parent 2_Année_N                

In [5]:
data['Age au bac'].fillna(0,inplace=True)
data['Bac - Mention (lib.)'].fillna("Sans mention",inplace=True)
data['Bac - Etab. préparation (code)'].fillna(0,inplace=True)
data['Code commune Lycée'].fillna(0,inplace=True)
data['IPS du lycée'].fillna(0,inplace=True)
data['PCS - autre parent (code)_Année_N'].fillna(0,inplace=True)
data['Profession du parent 2_Année_N'].fillna("NonRenseigné",inplace=True)
data['Adresse fixe - Commune (code)_Année_N'].fillna(0,inplace=True)
data['Adresse fixe - Commune (code)_Année_N+1'].fillna(0,inplace=True)

In [6]:
np.isnan(data['Résultat Session 1_Année_N'][2])==False

False

Code pour associer chaque note à une mention au niveau des deux sessions. Entre 0 et 10 la mention "Ajourné" sera associée et pour une note de plus de 10 la mention "Admis" sera associée. Le but de cette opération est de réduire la taille des valeurs manquantes dans les 4 colonnes.

In [7]:
for i in range(len(data)):
  if (data['Note Session 1_Année_N'][i]<10) & (data['Note Session 1_Année_N'][i]>=0) and pd.isnull(data['Résultat Session 1_Année_N'][i]) :
    data['Résultat Session 1_Année_N'][i]="Ajourné"
  if (data['Note Session 2_Année_N'][i]<10) & (data['Note Session 2_Année_N'][i]>=0 and pd.isnull(data['Résultat Session 2_Année_N'][i])):
    data['Résultat Session 2_Année_N'][i]="Ajourné"
  if (data['Note Session 1_Année_N'][i]>=10 and pd.isnull(data['Résultat Session 1_Année_N'][i])):
    data['Résultat Session 1_Année_N'][i]="Admis"
  if (data['Note Session 2_Année_N'][i]>=10 and pd.isnull(data['Résultat Session 2_Année_N'][i])):
    data['Résultat Session 2_Année_N'][i]="Admis"

<ipython-input-7-afbde0ba3788>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Résultat Session 2_Année_N'][i]="Admis"
<ipython-input-7-afbde0ba3788>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Résultat Session 1_Année_N'][i]="Admis"
<ipython-input-7-afbde0ba3788>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Résultat Session 2_Année_N'][i]="Ajourné"
<ipython-input-7-afbde0ba3788>:3: SettingWithCopyWarning: 


Dans le code suivant, si un étudiant a plus de 10 pour la session 1 et n'a pas de note pour la session 2, la note de la session 1 sera affectée à la note de la session 2. Nous considérons donc que si un étudiant a la moyenne pour la session 1 et n'a pas passé la session 2, la même note est conservée. De même si un étudiant a été admis ou admissible à la session 1, alors il est noté comme admis/admissible pour la session 2. Ceci est une continuité du code précédent pour le traitement des valeurs manquantes.

In [8]:
for i in range(len(data)):
  if data['Note Session 1_Année_N'][i]>=10 and pd.isnull(data['Note Session 2_Année_N'][i]):
    data['Note Session 2_Année_N'][i]=data['Note Session 1_Année_N'][i]
  if data['Résultat Session 1_Année_N'][i]!="" and pd.isnull(data['Résultat Session 2_Année_N'][i]):
    data['Résultat Session 2_Année_N'][i]=data['Résultat Session 1_Année_N'][i]


<ipython-input-8-a17d757997cd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Note Session 2_Année_N'][i]=data['Note Session 1_Année_N'][i]
<ipython-input-8-a17d757997cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Résultat Session 2_Année_N'][i]=data['Résultat Session 1_Année_N'][i]


In [9]:
for i in range(len(data)):
  if data['Note Session 1_Année_N'][i]<10 and pd.isnull(data['Note Session 2_Année_N'][i]):
    data['Note Session 2_Année_N'][i]=data['Note Session 1_Année_N'][i]
  if data['Note Session 2_Année_N'][i]<10 and pd.isnull(data['Résultat Session 2_Année_N'][i]):
    data['Résultat Session 2_Année_N'][i]=data['Résultat Session 1_Année_N'][i]

<ipython-input-9-ef0cc602bcdf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Note Session 2_Année_N'][i]=data['Note Session 1_Année_N'][i]


In [10]:
data.iloc[9736]
df15=data[data['RNA']=="RNA"]

df15['Note Session 1_Année_N'].max()

13.165

In [11]:
df15['Note Session 2_Année_N'].max()

13.367

In [12]:
file='don.xlsx'
data.to_excel(file)
print("Success")

Success


In [13]:
data['Note Session 1_Année_N'].fillna(-1,inplace=True)
data['Note Session 2_Année_N'].fillna(-1,inplace=True)

In [14]:
data['RNA'].value_counts()['RNA']/len(data)

0.10614997444064331

In [15]:
result = data.isna().sum().to_string(max_rows=None)
print(result)

Année_N                                            0
Individu - Code Etudiant                           0
Individu - Sexe                                    0
Nationalité (code)                                 0
Nationalité (lib.)                                 0
Nationalité - continent                            0
Age au bac                                         0
Bac ou équivalence (code)                          0
Groupe de bac                                      0
Série du bac                                       0
Bac - Mention (lib.)                               0
Bac - Etab. préparation (code)                     0
Code commune Lycée                                 0
IPS du lycée                                       0
Etudiant oui ou oui-si                             0
PCS - parent (code)_Année_N                        0
PCS - parent (lib.)_Année_N                        0
PCS - autre parent (code)_Année_N                  0
Profession du parent 2_Année_N                

In [16]:
mask = (
    pd.isnull(data['Note Session 1_Année_N']) &
    pd.isnull(data['Note Session 2_Année_N']) &
    pd.isnull(data['Résultat Session 1_Année_N']) &
    pd.isnull(data['Résultat Session 2_Année_N'])
)
#value = data[mask].iloc[2]['Note Session 1_Année_N']


In [17]:
data1=data[~mask]

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [19]:
data1.isna().sum()

Année_N                                            0
Individu - Code Etudiant                           0
Individu - Sexe                                    0
Nationalité (code)                                 0
Nationalité (lib.)                                 0
Nationalité - continent                            0
Age au bac                                         0
Bac ou équivalence (code)                          0
Groupe de bac                                      0
Série du bac                                       0
Bac - Mention (lib.)                               0
Bac - Etab. préparation (code)                     0
Code commune Lycée                                 0
IPS du lycée                                       0
Etudiant oui ou oui-si                             0
PCS - parent (code)_Année_N                        0
PCS - parent (lib.)_Année_N                        0
PCS - autre parent (code)_Année_N                  0
Profession du parent 2_Année_N                

In [20]:
#H=data[~data['Note Session 1_Année_N'].isna()]
H=data[~data['Résultat Session 1_Année_N'].isna()]

Pour le traitement des valeurs manquantes des notes, j'ai considéré les étudiants qui avaient au moins une note lors des deux sessions ou au moins un résultat(défaillant ou autre)

In [21]:
H.reset_index(drop=True, inplace=True)


In [22]:
for i in range(len(H)):
  if (H['Type d\'étape_Année_N'][i]=="M1"):
    H['Niveau_Année_N'][i]="bac+4"
    H['Cursus LMD_Année_N'][i]="cursus M"
  if H['Type d\'étape_Année_N+1'][i]=="M1":
    H['Niveau_Année_N+1'][i]="bac+4"
    H['Cursus LMD_Année_N+1'][i]="cursus M"
  if H['Type d\'étape_Année_N'][i]=="M2":
    H['Niveau_Année_N'][i]="bac+5"
    H['Cursus LMD_Année_N'][i]="cursus M"
  if H['Type d\'étape_Année_N+1'][i]=="M2":
    H['Niveau_Année_N+1'][i]="bac+5"
    H['Cursus LMD_Année_N+1'][i]="cursus M"
  if H['Type d\'étape_Année_N'][i]=="MEEF2":
    H['Niveau_Année_N'][i]="bac+5"
    H['Cursus LMD_Année_N'][i]="cursus M"
  if H['Type d\'étape_Année_N+1'][i]=="MEEF2":
    H['Niveau_Année_N+1'][i]="bac+5"
    H['Cursus LMD_Année_N+1'][i]="cursus M"



<ipython-input-22-b32c0372a5a2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Niveau_Année_N'][i]="bac+5"
<ipython-input-22-b32c0372a5a2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Cursus LMD_Année_N'][i]="cursus M"
<ipython-input-22-b32c0372a5a2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Niveau_Année_N+1'][i]="bac+5"
<ipython-input-22-b32c0372a5a2>:13: SettingWithCopyWarning: 
A value is trying to be set on 

In [23]:
H['Cursus LMD_Année_N'].fillna("cursus L",inplace=True)
H['Cursus LMD_Année_N+1'].fillna("cursus L",inplace=True)

<ipython-input-23-85710142c9f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Cursus LMD_Année_N'].fillna("cursus L",inplace=True)
<ipython-input-23-85710142c9f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Cursus LMD_Année_N+1'].fillna("cursus L",inplace=True)


Nous avons classé les étudiants en MEEF comme étant en master et de niveau bac+4/bac+5. Après cette manipulation il restait quelques valeurs manquantes au niveau du cursus. Après vérification seuls les étudiants en Licence Pro avait des valeurs manquantes au niveau de la variable "Cursus LMD" . Nous avons donc complété cette variable chez cette catégorie d'étudiant.

In [24]:
H['nom_de_la_commune_Année_N'].fillna("Etranger",inplace=True)
H['Distance_en_km_Année_N'].fillna(H['Distance_en_km_Année_N'].mean(),inplace=True)
H['nom_de_la_commune_Année_N+1'].fillna("Etranger",inplace=True)
H['Distance_en_km_Année_N+1'].fillna(H['Distance_en_km_Année_N+1'].mean(),inplace=True)

<ipython-input-24-6d091ed7986b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['nom_de_la_commune_Année_N'].fillna("Etranger",inplace=True)
<ipython-input-24-6d091ed7986b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['Distance_en_km_Année_N'].fillna(H['Distance_en_km_Année_N'].mean(),inplace=True)
<ipython-input-24-6d091ed7986b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  H['nom_de_la_commune_Année_N+1'].fillna("Etranger

Nous avons utilisé le fichier recensant les noms des communes et les coordonnees geographiques de ces communes sur le site de datagouv. N'ayant pas retrouvé certains codes communes dans ce fichier, nous allons considérer que ces communes se trouvent hors de la france, ce qui justifie le choix pour le remplissage des valeurs manquantes de ces variables. Pour la variable de la distance, nous avons choisi une imputation par la moyenne.

In [25]:
H.isna().sum()

Année_N                                            0
Individu - Code Etudiant                           0
Individu - Sexe                                    0
Nationalité (code)                                 0
Nationalité (lib.)                                 0
Nationalité - continent                            0
Age au bac                                         0
Bac ou équivalence (code)                          0
Groupe de bac                                      0
Série du bac                                       0
Bac - Mention (lib.)                               0
Bac - Etab. préparation (code)                     0
Code commune Lycée                                 0
IPS du lycée                                       0
Etudiant oui ou oui-si                             0
PCS - parent (code)_Année_N                        0
PCS - parent (lib.)_Année_N                        0
PCS - autre parent (code)_Année_N                  0
Profession du parent 2_Année_N                

In [26]:
file1='don1.xlsx'
H.to_excel(file1)
print("Success")

Success


In [27]:
# Identifions les colonnes de type "object"
object_cols = data.select_dtypes(include=['object']).columns

# Remplacons les valeurs manquantes dans ces colonnes
data[object_cols] = data[object_cols].fillna("NonRenseigné")

In [28]:
np.isnan(data['Note Session 1_Année_N'][2]) & np.isnan(data['Note Session 2_Année_N'][2])

False

In [29]:
H.isna().sum()

Année_N                                            0
Individu - Code Etudiant                           0
Individu - Sexe                                    0
Nationalité (code)                                 0
Nationalité (lib.)                                 0
Nationalité - continent                            0
Age au bac                                         0
Bac ou équivalence (code)                          0
Groupe de bac                                      0
Série du bac                                       0
Bac - Mention (lib.)                               0
Bac - Etab. préparation (code)                     0
Code commune Lycée                                 0
IPS du lycée                                       0
Etudiant oui ou oui-si                             0
PCS - parent (code)_Année_N                        0
PCS - parent (lib.)_Année_N                        0
PCS - autre parent (code)_Année_N                  0
Profession du parent 2_Année_N                